<a href="https://colab.research.google.com/github/driesvr/driesvr.github.io/blob/main/notebooks/free_wilson_cornercases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Free Wilson Analysis: addressing some corner cases

### Introduction
In this notebook we'll try to address some corner cases that may come up when running Free Wilson Analyses with RDKit. The base script is is taken directly from Pat Wilson's excellent Free-Wilson Analysis notebook, and we'll be making just a few modifications to remedy issues with specific types of molecules. I've taken the liberty of condensing Pat's notebook to the bare essentials needed to run FWA, I would consider the full notebook which you can find [here](https://colab.research.google.com/github/PatWalters/practical_cheminformatics_tutorials/blob/main/sar_analysis/free_wilson.ipynb) to be essential reading for cheminformaticians.

In [1]:
!pip install pandas mols2grid scikit-learn seaborn useful_rdkit_utils numpy

In [2]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem.rdRGroupDecomposition import RGroupDecompose
from rdkit.Chem.TemplateAlign import AlignMolToTemplate2D
from rdkit.Chem.rdDepictor import Compute2DCoords
from rdkit.Chem import AllChem
import mols2grid
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
import seaborn as sns
from ipywidgets import interact
from itertools import product
import useful_rdkit_utils as uru
from tqdm.auto import tqdm
import numpy as np

Let's add in a few molecules that can cause issues in the default notebook. Fused systems as well as doubly substituted systems have the potential to cause trouble, as we will see below. I'm giving them excellent pIC50s so we can easily find these back later on among the top performers.

In [3]:
additional_mols = pd.DataFrame({'SMILES':['CC(C)OC(=O)C1C(c2ccc(C3CC)c(C3)c2)CC2CCC1N2C',
                                          'CC(C)OC(=O)C(C)1C(c2ccccc2)CC2CCC1N2C',
                                          'CC(C)1C(c2ccccc2)CC2CCC1N2C'],
                                'Name': ['Fused system',
                                         'Double substituted 1',
                                         'Double substituted 2',],
                                'pIC50' : [11,
                                           11,
                                           11],
})
mols2grid.display(additional_mols, subset = ['Name'], size=(300,200))

MolGridWidget()

In [4]:
input_filename = "https://raw.githubusercontent.com/PatWalters/practical_cheminformatics_tutorials/main/data/CHEMBL313_sel.smi"
df = pd.read_csv(input_filename).sample(30)
df = pd.concat([df,additional_mols])
df['mol'] = df.SMILES.apply(Chem.MolFromSmiles)


Note that we are subsampling the dataset to make things run faster above. Next, specify the core and note that R-groups don't have to be labeled.

In [5]:
core_smiles = "c1ccc(C2CC3CCC(C2)N3)cc1"
core_mol = Chem.MolFromSmiles(core_smiles)

### R-Group Decomposition
Perform the R-group decomposition. Here's the first of our modifications. By setting allowMultipleRGroupsOnUnlabelled to be True, we separate out doubly substituted sites into two distinct R-groups.

In [6]:
from rdkit.Chem import rdRGroupDecomposition
ps = rdRGroupDecomposition.RGroupDecompositionParameters()
ps.allowMultipleRGroupsOnUnlabelled = True

match, miss = RGroupDecompose(core_mol,df.mol.values,asSmiles=True, options=ps)

print(f"{len(miss)} molecules were not matched")
rgroup_df = pd.DataFrame(match)
rgroup_df

0 molecules were not matched


,Core,R1,R2,R3,R4,R5
0,c1cc([*:5])c([*:4])cc1C1CC2CCC(N2[*:1])C1([*:2...,C[*:1],[H][*:2],COC(=O)[*:3],[H][*:4],c1coc([*:5])c1
1,c1cc([*:5])c([*:4])cc1C1CC2CCC(N2[*:1])C1([*:2...,BrCCC[*:1],[H][*:2],COC(=O)[*:3],[H][*:4],I[*:5]
2,c1cc([*:5])c([*:4])cc1C1CC2CCC(N2[*:1])C1([*:2...,C[*:1],[H][*:2],COC(=O)[*:3],I[*:4],[H][*:5]
3,c1cc([*:5])c([*:4])cc1C1CC2CCC(N2[*:1])C1([*:2...,FCCC[*:1],[H][*:2],COC(=O)[*:3],[H][*:4],C[*:5]
4,c1cc([*:5])c([*:4])cc1C1CC2CCC(N2[*:1])C1([*:2...,[H][*:1],[H][*:2],COC(=O)[*:3],F[*:4],[H][*:5]
5,c1cc([*:5])c([*:4])cc1C1CC2CCC(N2[*:1])C1([*:2...,C[*:1],[H][*:2],CCC(=O)[*:3],[H][*:4],CC(C)(C)[*:5]
6,c1cc([*:5])c([*:4])cc1C1CC2CCC(N2[*:1])C1([*:2...,FCC[*:1],[H][*:2],COC(=O)[*:3],[H][*:4],C[*:5]
7,c1cc([*:5])c([*:4])cc1C1CC2CCC(N2[*:1])C1([*:2...,[H][*:1],[H][*:2],CCC(=O)[*:3],[H][*:4],I/C=C/[*:5]
8,c1cc([*:5])c([*:4])cc1C1CC2CCC(N2[*:1])C1([*:2...,C[*:1],[H][*:2],COC(=O)[*:3],[H][*:4],Clc1ccc([*:5])s1
9,c1cc([*:5])c([*:4])cc1C1CC2CCC(N2[*:1])C1([*:2...,[H][*:1],[H][*:2],COC(=O)[*:3],[H][*:4],I[*:5]


Display the core(s) found by the R-group decomposition, noting that the fused pyrrolidine now has R3 and R4 arising from the same carbon.

In [7]:
core_df = pd.DataFrame({"mol" : [Chem.MolFromSmiles(x) for x in rgroup_df.Core.unique()]})
mols2grid.display(core_df,size=(300,200),mol_col="mol")

MolGridWidget()

### Collect Unique R-Groups
Create lists of unique R-groups

In [8]:
unique_list = []
for r in rgroup_df.columns[1:]:
    num_rgroups = len(rgroup_df[r].unique())
    print(r,num_rgroups)
    unique_list.append(rgroup_df[r].unique())
total_possible_products = np.prod([len(x) for x in unique_list])
print(f"{total_possible_products:,} products possible")

R1 7
R2 2
R3 13
R4 7
R5 12
15,288 products possible


### Build a Ridge Regression Model

Use a one hot encoder to encode R-groups into a bit string

In [9]:
enc = OneHotEncoder(categories=unique_list,sparse_output=False)
one_hot_mat = enc.fit_transform(rgroup_df.values[:,1:])

### Generate and Evaluate R-Group Combinations

Now we'll build all combinations of the R-groups and use our model to predict their activity. In order to have the most effective model, we'll build the model with all the available data.

In [10]:

full_model = Ridge()
full_model.fit(one_hot_mat,df.pIC50)
already_made_smiles = set([Chem.MolToSmiles(x) for x in df.mol])

I've rewritten Pat's code here to print out substituents where it fails.

In [11]:
uru.rd_shut_the_hell_up()
prod_list = []
for i,p in tqdm(enumerate(product(*enc.categories)),total=total_possible_products):
    core_smiles = rgroup_df.Core.values[0]
    try:
      smi = (".".join(p))
      mol = Chem.MolFromSmiles(smi+"."+core_smiles)
      prod = Chem.molzip(mol)
      prod = Chem.RemoveAllHs(prod)
      prod_smi = Chem.MolToSmiles(prod)
      if prod_smi not in already_made_smiles:
          desc = enc.transform([p])
          prod_pred_ic50 = full_model.predict(desc)[0]
          prod_list.append([prod_smi,prod_pred_ic50])
    except:
        print(p)
        break





  0%|          | 0/15288 [00:00<?, ?it/s]

('C[*:1]', '[H][*:2]', 'COC(=O)[*:3]', '[H][*:4]', 'CCC(C[*:4])[*:5]')


The fused system connects on the R1 and R5 positions. Unfortunately, this clashes with the H substituent already in place on the R1 position. This is a molecule that can't really exist, so we would like to skip it. I'm introducing some code to check for duplicate numbers, skipping those molecules.

In [12]:
import re
def has_shared_number(string_tuple):
  """Checks if any number within the [*:number] format is shared among strings in a tuple.

  Args:
    string_tuple: A tuple containing strings with potential [*:number] patterns.

  Returns:
    True if any number is shared, False otherwise.
  """
  all_numbers = []
  for string in string_tuple:
    numbers = [int(match.group(1)) for match in re.finditer(r"\[\*:(\d+)\]", string)]
    all_numbers.extend(numbers)
  return len(set(all_numbers)) < len(all_numbers)


prod_list = []
for i,p in tqdm(enumerate(product(*enc.categories)),total=total_possible_products):
    core_smiles = rgroup_df.Core.values[0]
    if has_shared_number(p):
      continue

    try:
      smi = (".".join(p))
      mol = Chem.MolFromSmiles(smi+"."+core_smiles)
      prod = Chem.molzip(mol)
      prod = Chem.RemoveAllHs(prod)
      prod_smi = Chem.MolToSmiles(prod)
      if prod_smi not in already_made_smiles:
          desc = enc.transform([p])
          prod_pred_ic50 = full_model.predict(desc)[0]
          prod_list.append([prod_smi,prod_pred_ic50])
    except:
        print(p)
        break



  0%|          | 0/15288 [00:00<?, ?it/s]

In [13]:
prod_df = pd.DataFrame(prod_list,columns=["SMILES","Pred_pIC50"])
prod_df.sort_values("Pred_pIC50",ascending=False,inplace=True)
best_df = prod_df.head(100).copy()
best_df['mol'] = best_df.SMILES.apply(Chem.MolFromSmiles)
mols2grid.display(best_df,mol_col='mol',prerender=True, substruct_highlight=False,
                  transform={"Pred_pIC50" : lambda x: f"{x:.1f}"},
                  subset=["img","Pred_pIC50"])

MolGridWidget()

This is a step in the right direction - it at least doesn't crash - but as you can see it also filters out the fused systems, which have the string `CCC(C[*:1])[*:5]` in both R1 and R5. We can get those back by modifying our code a bit to check for and remove duplicate strings in the R-groups. This won't affect doubly substituted systems with identical R-groups, as we have set up the R-group decomposition to assign them to different R-group indices.

In [14]:

prod_list = []
for i,p in tqdm(enumerate(product(*enc.categories)),total=total_possible_products):
    core_smiles = rgroup_df.Core.values[0]
    if has_shared_number(p):
      if len(set(p)) != len(p):
        smi = (".".join(list(set(p))))
      else:
        continue
    else:
      smi = (".".join(p))
    mol = Chem.MolFromSmiles(smi+"."+core_smiles)
    prod = Chem.molzip(mol)
    prod = Chem.RemoveAllHs(prod)
    prod_smi = Chem.MolToSmiles(prod)
    if prod_smi not in already_made_smiles:
        desc = enc.transform([p])
        prod_pred_ic50 = full_model.predict(desc)[0]
        prod_list.append([prod_smi,prod_pred_ic50])



  0%|          | 0/15288 [00:00<?, ?it/s]

Put the generated molecules into a dataframe.

In [17]:
prod_df = pd.DataFrame(prod_list,columns=["SMILES","Pred_pIC50"])
prod_df.sort_values("Pred_pIC50",ascending=False,inplace=True)
best_df = prod_df.head(100).copy()
best_df['mol'] = best_df.SMILES.apply(Chem.MolFromSmiles)
mols2grid.display(best_df,mol_col='mol',prerender=True, substruct_highlight=False,
                  transform={"Pred_pIC50" : lambda x: f"{x:.1f}"},
                subset=["img","Pred_pIC50"], size=(300,200))

MolGridWidget()

There we go, it seems all corner cases that we set out to cover are now working!